# faceRec

This IPython notebook will recognize a face in a video

# Installation

The following was installed on OSX 10.10.5 (Yosemite)

### Homebrew

Install homebrew (http://brew.sh/) if not already installed

**Homebrew maintenance**<br>
```$ brew doctor
$ brew update
$ brew upgrade```

### OpenCV
```$ brew tap homebrew/science
$ brew install opencv```

### Numpy
```$ pip install numpy```

### Python path

**Edit your .bash_profile in your home directory to include:**<br> 
```export PYTHONPATH=/usr/local/lib/python2.7/site-packages:$PYTHONPATH```

**Save and reload the bash profile**<br>
```$ source .bash_profile```

# Import the required modules

In [ ]:
# import the required modules
import cv2
import numpy
import math
import os

In [ ]:
# Get user supplied values
imagePath = '/Users/abock/Downloads/Barack_Obama_family_portrait_2011.jpg'
cascPath = '/usr/local/Cellar/opencv/2.4.13.2/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml'

In [ ]:
# Create the haar cascade
faceCascade = cv2.CascadeClassifier(cascPath)

In [ ]:
# Read the image
image = cv2.imread(imagePath)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
imS = cv2.resize(image, (960, 540))

In [ ]:
cv2.imshow('raw',imS)

In [ ]:
cv2.destroyAllWindows()

# Download images from Google

In [ ]:
from bs4 import BeautifulSoup
import urllib2 
import urllib

def saveURLimages(url):
	req = urllib2.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
	html = urllib2.urlopen(req).read()
	soup = BeautifulSoup(html)
	# get images
	images = [img for img in soup.findAll('img')]
	print (str(len(images)) + " images found.")
	print 'Downloading images to current working directory.'
	
	# download images
	image_links = [each.get('src') for each in images]
	for each in image_links:
  		filename=each.split('/')[-1]
  		urllib.urlretrieve(each,filename)

In [ ]:
url = "https://www.google.com/search?q=happy+people&rlz=1C5AVSZ_enUS650US650&espv=2&biw=840&bih=949&source=lnms&tbm=isch&sa=X&ved=0ahUKEwiVz-ehp8fRAhUJ6GMKHeYaA4QQ_AUIBigB"
saveURLimages(url)